In [1]:
import cycpd
import matplotlib.pyplot as plt
import pycpd
import numpy as np
import pandas as pd
import os
import glob
import shutil
import open3d as o3d
import time
import h5py
import copy
import trimesh
from utils import rotationMatrixToEulerAngles, fixedNumDownSample, voxelDownSample, showPointCloud, surfaceVertices2WatertightO3dMesh, loadAlignedPointGroupsWithIndex, getEigValVecOfSSMByPCA

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
DATA_ROOT_PATH = r".\dataWithPhoto"
SRC_DATA_PATH = r".\dataWithPhoto\normal"

In [3]:
def __tagFromFileName(f):
    return int(os.path.basename(f).split("_")[1])

def getSrcDataInfoDF(srcDataDir):
    lowerScanObjs = glob.glob(os.path.join(srcDataDir,"*_lower.obj"))
    lowerVertexLabelTxts = glob.glob(os.path.join(srcDataDir,"*_lower.txt"))
    lowerAuxilTxts = glob.glob(os.path.join(srcDataDir,"*_lowerFDI.txt"))
    lowerRefinedTeethObjs = glob.glob(os.path.join(srcDataDir,"*_lowerTooth.obj"))
    upperScanObjs = glob.glob(os.path.join(srcDataDir,"*_upper.obj"))
    upperVertexLabelTxts = glob.glob(os.path.join(srcDataDir,"*_upper.txt"))
    upperAuxilTxts = glob.glob(os.path.join(srcDataDir,"*_upperFDI.txt"))
    upperRefinedTeethObjs = glob.glob(os.path.join(srcDataDir,"*_upperTooth.obj"))
    lowerPhotos = glob.glob(os.path.join(srcDataDir,"*_下牙列.jpg"))
    upperPhotos = glob.glob(os.path.join(srcDataDir,"*_上牙列.jpg"))
    leftPhotos = glob.glob(os.path.join(srcDataDir,"*_左侧咬合.jpg"))
    rightPhotos = glob.glob(os.path.join(srcDataDir,"*_右侧咬合.jpg"))
    frontalPhotos = glob.glob(os.path.join(srcDataDir,"*_正位像.jpg"))
    
    lowerScanObjs = sorted(lowerScanObjs, key=lambda x:__tagFromFileName(x))
    lowerVertexLabelTxts = sorted(lowerVertexLabelTxts, key=lambda x:__tagFromFileName(x))
    lowerAuxilTxts = sorted(lowerAuxilTxts, key=lambda x:__tagFromFileName(x))
    lowerRefinedTeethObjs = sorted(lowerRefinedTeethObjs, key=lambda x:__tagFromFileName(x))
    upperScanObjs = sorted(upperScanObjs, key=lambda x:__tagFromFileName(x))
    upperVertexLabelTxts = sorted(upperVertexLabelTxts, key=lambda x:__tagFromFileName(x))
    upperAuxilTxts = sorted(upperAuxilTxts, key=lambda x:__tagFromFileName(x))
    upperRefinedTeethObjs = sorted(upperRefinedTeethObjs, key=lambda x:__tagFromFileName(x))
    lowerPhotos = sorted(lowerPhotos, key=lambda x:__tagFromFileName(x))
    upperPhotos = sorted(upperPhotos, key=lambda x:__tagFromFileName(x))
    leftPhotos = sorted(leftPhotos, key=lambda x:__tagFromFileName(x))
    rightPhotos = sorted(rightPhotos, key=lambda x:__tagFromFileName(x))
    frontalPhotos = sorted(frontalPhotos, key=lambda x:__tagFromFileName(x))
    
    
    tags = [__tagFromFileName(x) for x in lowerScanObjs]
    indices = np.arange(len(tags),dtype=np.uint32)
    infoDF = pd.DataFrame({"index":indices, "tag":tags, "lowerScanObj":lowerScanObjs, "lowerVertexLabelTxt":lowerVertexLabelTxts, "lowerAuxilTxt":lowerAuxilTxts, 
                           "lowerRefinedTeethObj":lowerRefinedTeethObjs, "upperScanObj":upperScanObjs, "upperVertexLabelTxt":upperVertexLabelTxts,
                           "upperAuxilTxt":upperAuxilTxts, "upperRefinedTeethObj":upperRefinedTeethObjs, "lowerPhoto":lowerPhotos, 
                           "upperPhoto":upperPhotos, "leftPhoto":leftPhotos, "rightPhoto":rightPhotos, "frontalPhoto":frontalPhotos})
    return infoDF

In [5]:
initInfoDF = getSrcDataInfoDF(SRC_DATA_PATH)
initInfoDF.to_csv(os.path.join(DATA_ROOT_PATH,"nameIndexMapping.csv"), index=False, encoding='"utf_8_sig"')

In [4]:
def getLargestConnectedO3dMeshComponent(o3dMesh):
    """ 获取三角面片中的最大连通区域"""
    __mesh = copy.deepcopy(o3dMesh)
    triangle_clusters, cluster_n_triangles, cluster_area = (__mesh.cluster_connected_triangles())
    triangle_clusters = np.asarray(triangle_clusters)
    cluster_n_triangles = np.asarray(cluster_n_triangles)
    # print("Number of triangle clusters: ", len(cluster_n_triangles))
    # print("triangle number in clusters: ", cluster_n_triangles)
    cluster_area = np.asarray(cluster_area)
    largest_cluster_idx = cluster_n_triangles.argmax()
    triangles_to_remove = triangle_clusters != largest_cluster_idx
    goodConnectionFlag = np.sum(cluster_n_triangles) < 1.2*cluster_n_triangles[largest_cluster_idx]
    __mesh.remove_triangles_by_mask(triangles_to_remove)
    __mesh.remove_unreferenced_vertices()
    return __mesh, goodConnectionFlag

def removeO3dMeshSmallComponents(o3dMesh, minNumVertex2Keep):
    __mesh = copy.deepcopy(o3dMesh)
    triangle_clusters, cluster_n_triangles, cluster_area = (__mesh.cluster_connected_triangles())
    triangle_clusters = np.asarray(triangle_clusters)
    cluster_n_triangles = np.asarray(cluster_n_triangles)
    # print("Number of triangle clusters: ", len(cluster_n_triangles))
    # print("triangle number in clusters: ", cluster_n_triangles)
    cluster_area = np.asarray(cluster_area)
    triangles_to_remove = cluster_n_triangles[triangle_clusters] < minNumVertex2Keep
    __mesh.remove_triangles_by_mask(triangles_to_remove)
    __mesh.remove_unreferenced_vertices()
    return __mesh

def separateScanMesh(scanObj, vertexLabelTxt, auxilTxt):
    """按照顶点标签分割口扫obj的每颗牙齿，返回每颗牙齿的opend3d.geometry.TriangleMesh"""
    scanMesh = o3d.io.read_triangle_mesh(scanObj)
    vertexLabels = np.loadtxt(vertexLabelTxt, dtype=np.uint32)
    auxilIndices = np.loadtxt(auxilTxt, dtype=np.uint32)
    indexMap = np.hstack([np.array([0]), auxilIndices]) # mapping from 1-16 to 11-18,21-28 or 41-48,31-38
    vertexLabels = indexMap[vertexLabels]
    vertexIndices = np.arange(vertexLabels.shape[0], dtype=np.uint32)
    toothMeshes = {}
    for toothID in indexMap[1:]:
        selectedVertexIndices = vertexIndices[vertexLabels==toothID]
        toothMesh = scanMesh.select_by_index(selectedVertexIndices, cleanup=True)
        # toothMesh, goodConnectionFlag = getLargestConnectedO3dMeshComponent(toothMesh)
        toothMesh = removeO3dMeshSmallComponents(toothMesh, minNumVertex2Keep=200)
        toothMeshes[toothID] = toothMesh
    return toothMeshes

In [5]:
SEP_OBJ_SAVE_DIR = r".\dataWithPhoto\format-obj"
SEP_TXT_SAVE_DIR = r".\dataWithPhoto\format-txt"

In [4]:
# if os.path.exists(SEP_OBJ_SAVE_DIR):
#     shutil.rmtree(SEP_OBJ_SAVE_DIR)
# if os.path.exists(SEP_TXT_SAVE_DIR):
#     shutil.rmtree(SEP_TXT_SAVE_DIR)

In [5]:
def writeSeparatedToothMesh(toothMeshes, tag, objSaveDir, txtSaveDir):
    for toothIndex, tMesh in toothMeshes.items():
        objDir = os.path.join(objSaveDir, str(toothIndex))
        if not os.path.exists(objDir):
            os.makedirs(objDir)
        objF = os.path.join(objSaveDir, str(toothIndex), "{}.obj".format(tag))
        o3d.io.write_triangle_mesh(objF, tMesh, write_ascii=False, compressed=False, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False, print_progress=False)
        print("Finish writing ", objF)
        txtDir = os.path.join(txtSaveDir, str(toothIndex))
        if not os.path.exists(txtDir):
            os.makedirs(txtDir)
        txtF = os.path.join(txtSaveDir, str(toothIndex), "{}.txt".format(tag))
        np.savetxt(txtF, np.asarray(tMesh.vertices))
        print("Finish writing ", txtF)

In [12]:
# # 将口扫ScanMesh按牙齿编号分割，保存obj，txt
# for index, row in initInfoDF.iterrows():
#     tag = row["index"]
#     lowerToothMeshes = separateScanMesh(row["lowerScanObj"], row["lowerVertexLabelTxt"], row["lowerAuxilTxt"])
#     writeSeparatedToothMesh(lowerToothMeshes, str(tag)+"L", SEP_OBJ_SAVE_DIR, SEP_TXT_SAVE_DIR)
#     upperToothMeshes = separateScanMesh(row["upperScanObj"], row["upperVertexLabelTxt"], row["upperAuxilTxt"])
#     writeSeparatedToothMesh(upperToothMeshes, str(tag)+"U", SEP_OBJ_SAVE_DIR, SEP_TXT_SAVE_DIR)

Finish writing  .\dataWithPhoto\format-obj\47\0L.obj
Finish writing  .\dataWithPhoto\format-txt\47\0L.txt
Finish writing  .\dataWithPhoto\format-obj\46\0L.obj
Finish writing  .\dataWithPhoto\format-txt\46\0L.txt
Finish writing  .\dataWithPhoto\format-obj\45\0L.obj
Finish writing  .\dataWithPhoto\format-txt\45\0L.txt
Finish writing  .\dataWithPhoto\format-obj\44\0L.obj
Finish writing  .\dataWithPhoto\format-txt\44\0L.txt
Finish writing  .\dataWithPhoto\format-obj\43\0L.obj
Finish writing  .\dataWithPhoto\format-txt\43\0L.txt
Finish writing  .\dataWithPhoto\format-obj\42\0L.obj
Finish writing  .\dataWithPhoto\format-txt\42\0L.txt
Finish writing  .\dataWithPhoto\format-obj\41\0L.obj
Finish writing  .\dataWithPhoto\format-txt\41\0L.txt
Finish writing  .\dataWithPhoto\format-obj\31\0L.obj
Finish writing  .\dataWithPhoto\format-txt\31\0L.txt
Finish writing  .\dataWithPhoto\format-obj\32\0L.obj
Finish writing  .\dataWithPhoto\format-txt\32\0L.txt
Finish writing  .\dataWithPhoto\format-obj\33\

In [6]:
def getFileTag(f):
    return os.path.basename(f).split(".")[0]

In [7]:
def showPointClouds(p1,p2,tag):
    pcd1 = o3d.geometry.PointCloud()
    pcd1.points = o3d.utility.Vector3dVector(p1)
    pcd1.paint_uniform_color(np.array([1.,0.,0.]))
    pcd2 = o3d.geometry.PointCloud()
    pcd2.points = o3d.utility.Vector3dVector(p2)
    pcd2.paint_uniform_color(np.array([0.,0.,1.]))
    o3d.visualization.draw_geometries([pcd1,pcd2], window_name=tag, width=800, height=600, left=50,top=50, point_show_normal=False)
    
def removeOutliers(vertices, nb_points=2, radius=0.5):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(vertices)
    cl, ind = pcd.remove_radius_outlier(nb_points=nb_points, radius=radius)
    pcd = pcd.select_by_index(ind, invert=False)
    return np.asarray(pcd.points)

In [9]:
toothIndex = 11
scanPgDir = r".\dataWithPhoto\format-txt\{}".format(toothIndex)
repairedPgDir = r".\dataWithPhoto\repaired-txt\{}".format(toothIndex)
scanPgFiles = glob.glob(os.path.join(scanPgDir,"*.txt"))
scanPgFiles = sorted(scanPgFiles, key=lambda x:int(getFileTag(x)[:-1]))
repairedPgFiles = glob.glob(os.path.join(repairedPgDir,"*.txt"))
repairedPgFiles = sorted(repairedPgFiles, key=lambda x:int(getFileTag(x)[:-1]))

scanPGs = [np.loadtxt(f) for f in scanPgFiles]
repairedPGs = [np.loadtxt(f) for f in repairedPgFiles]

In [193]:
# 检查修复的所有牙齿点云
for scanPG,predRepairedPG,tag in zip(scanPGs, repairedPGs, repairedPgFiles):
    showPointClouds(scanPG,predRepairedPG, tag)

In [10]:
IndicesOfScanPgRepairedBySSM = {11:[15,18,22,25,37,40,55,58,59,68,73,83,89,91],
                      12:[31,36,61,68,69],
                      13:[0,22,35,45,58,65,68,77,89], 
                      14:[12,31,35,54,63,69,77],
                      15:[2,12,26,52,65,74], 
                      16:[2,14,47,50,51,54,63,86],
                      17:[32,33,64,74,81,86,93],
                      21:[6,15,18,20,24,37,40,62,85,89,91],
                      22:[6,15,25,34,48,49,56,59,80],
                      23:[7,20,38,41,49,57,75],
                      24:[12,23,30,38,48,60,75,82,84,89],
                      25:[3,23,29,38,42,60,76,79,84],
                      26:[3,9,16,19,27,28,29,42,46,72,87,88,90,92],
                      27:[1,9,16,21,27,29,39,46,66,72,87,88,94],
                      31:[23,25,28,48,53,55,58,75,90,93],
                      32:[4,25,34,52,90,93],
                      33:[1,25,26,52],
                      34:[34,54,69],
                      35:[26,29,44,54,56,69,71,76],
                      36:[2,27,30,33,38,47,49,56,58,71,73,86,94],
                      37:[14,16,17,27,30,38,47,49,50,51,72,84,90,94],
                      41:[10,19,23,59,62,75,80,81],
                      42:[3,12,19,32,58,64],
                      43:[7,8,12,19,24,41,57,59,64],
                      44:[40,41,61,76,80],
                      45:[0,13,37,40,46,61,66,76,79],
                      46:[0,5,15,35,36,37,40,42,45,48,66,67,68,70,77,79,85],
                      47:[13,21,22,35,36,45,48,60,63,68,70,83,92]}

In [11]:
TagsOfScanPgRepairedBySSM = {k:["{}{}".format(id,"U") if k < 30 else "{}{}".format(id,"L") for id in v] for k,v in IndicesOfScanPgRepairedBySSM.items()}

In [15]:
# 检查使用SSM修复的牙齿数据
toothIndex = 11
scanPgDir = r".\dataWithPhoto\format-txt\{}".format(toothIndex)
repairedPgDir = r".\dataWithPhoto\repaired-txt\{}".format(toothIndex)
fileTags = TagsOfScanPgRepairedBySSM[toothIndex] # 需要用SSM修复的tag
scanPgTxts = [os.path.join(scanPgDir,tag+".txt") for tag in fileTags]
scanPGs = [np.loadtxt(x) for x in scanPgTxts]
repairedPgTxts = [os.path.join(repairedPgDir, tag+".txt") for tag in fileTags]
repairedPGs = [np.loadtxt(x) for x in repairedPgTxts]
print("{}: {}".format(toothIndex, fileTags))

11: ['15U', '18U', '22U', '25U', '37U', '40U', '55U', '58U', '59U', '68U', '73U', '83U', '89U', '91U']


In [16]:
for scanPG,predRepairedPG,tag in zip(scanPGs, repairedPGs, repairedPgTxts):
    showPointClouds(scanPG,predRepairedPG, tag)

In [15]:
def saveEigValVec(srcRootDir=r"./data/cpdAlignedData", NumPC2Save=100):
    upperToothIndices = [11,12,13,14,15,16,17,21,22,23,24,25,26,27] #不考虑智齿18,28
    lowerToothIndices = [31,32,33,34,35,36,37,41,42,43,44,45,46,47]
    toothIndices = upperToothIndices + lowerToothIndices
    for toothIndex in toothIndices:
        srcDir = os.path.join(srcRootDir, str(toothIndex))
        alignedPointGroups, alignedPgTags = loadAlignedPointGroupsWithIndex(srcDir)
        eigVal, eigVec, A, meanTrainPointVector = getEigValVecOfSSMByPCA(alignedPointGroups)
        meanAlignedPG = meanTrainPointVector.reshape(-1,3)
        eigVal = eigVal[:NumPC2Save]
        eigVec = eigVec[:,:NumPC2Save]
        saveDir = os.path.join(srcRootDir, "eigValVec")
        if not os.path.exists(saveDir):
            os.makedirs(saveDir)
        np.save(os.path.join(saveDir,"eigVal_{}.npy".format(toothIndex)), eigVal)
        np.save(os.path.join(saveDir,"eigVec_{}.npy".format(toothIndex)), eigVec)
        np.save(os.path.join(saveDir,"meanAlignedPG_{}.npy".format(toothIndex)), meanAlignedPG)

In [16]:
saveEigValVec(srcRootDir=r"./data/cpdAlignedData", NumPC2Save=100)